# API Server Test w/ Kafka
- API server에 요청을 보내고 결과를 확인할 수 있습니다.

## 주의사항
- __본 Notebook을 실행하기 전에 구현중인 모델 서버가 실행되어 있는지 확인해주세요.__
    - 모델 서버가 실행되어 있지 않다면, 아래 터미널에서 아래 명령어를 실행해주세요.<br>
    `python app.py -c examples/kafka-dev.conf`


- 실행 중 패키지 missing 및 임포트 문제가 있을 경우, 아래 명령어를 실행해주세요.<br>
  `uv add ipykernel requests` 또는 `pip install ipykernel requests`


- 본 Notebook 에 대한 문의사항은 아래 링크에 이슈를 생성하시고, `@Horeng`으로 assign 혹은 mention 해주세요.
    - [Github Issues](https://github.com/kyeru/detection-server/issues)


In [ ]:
# [[ 필수 실행 셀 ]]
# Auto reload imports
%load_ext autoreload
%autoreload 2

# Imports - IPython
from IPython.display import Markdown, display

# Imports - Standard
import sys
import json
import logging
import requests
from pathlib import Path

# Add project root to sys.path
PROJECT_ROOT = str(Path.cwd().parent)  #  `detection-server` 프로젝트 루트
print(f'{PROJECT_ROOT=}')
sys.path.append(PROJECT_ROOT)

# Logger
logger = logging.getLogger(__name__)
logger.info('test')

### NOTE: 아래 셀을 실행하기 전에 아래 변수를 수정해주세요.
`API_SERVER_IP`, `API_SERVER_PORT`

In [11]:
# [[ 필수 실행 셀 ]]
# Constants
ENCODING_UTF8 = 'utf-8'
####################
### [변경 필수!!!] ###
####################
API_SERVER_IP: str = 'API_SERVER_IP'  
API_SERVER_PORT: int | str = 'API_SERVER_PORT'
####################

api_server = f'http://{API_SERVER_IP}:{API_SERVER_PORT}'
api_endpoint_task_init = '{api_server}/api/v1/task/init'
api_doc_task_init = '{api_server}/swagger-ui/index.html#/plugin%20API%20v1/initDetection'
api_endpoint_task_progress = '{api_server}/api/v1/task/progress'
api_doc_task_progress = '{api_server}/swagger-ui/index.html#/plugin%20API%20v1/checkProgress'
model_name = 'SELab.Fixer'

# Support Methods
from kafka_utils import RequestMessage
from kafka_utils import ReportRecord

Token = str
def send_request_to_api(api_server: str, model_name: str, payload: dict) -> Token | Exception:
    # Prepare the data from the plugin
    request_message = RequestMessage(
        model_name=model_name, value=payload)

    # Send the data to the `api/v1/task/init` endpoint
    # global: api_endpoint_task_init
    task_init_api_endpoint = api_endpoint_task_init.format(api_server=api_server)
    try:
        response = requests.post(
            task_init_api_endpoint,
            headers={'Content-Type': 'application/json'},
            data=request_message.jsonify())  #  single-lined json format string
        token = response.text
    except Exception as e:
        return e
    
    return token

def get_progress_from_api(api_server: str, token: Token, verbose: bool = False) -> dict:
    assert api_server is not None, '`api_server` is required'
    assert token is not None, '`token` is required'
    response = requests.get(
        api_endpoint_task_progress.format(api_server=api_server),
        params=dict(token=token))
    if verbose:
        print(f'response: {response.json()}')
    report_record = ReportRecord(**response.json())
    return report_record

# 1. Sending a request to API server
- 아래 셀을 통해서 API server에 요청을 보낼 수 있습니다.
- Plugin에서 요청을 보내는 것과 동일한 방식입니다.
- `payload`의 데이터타입을 `dict`로만 유지하면 되며, 내부 항목은 자유롭게 입력해주세요.
- __이미 확보한 `token`이 존재한다면, 아래 셀을 실행하지 않고 바로 결과 확인 셀(Sec.2)을 실행해주세요.__


In [ ]:
# Send a request to the `request` topic through the `api/v1/task/init` endpoint

# 요청 데이터 준비
payload = dict(
    data='Hello World!'
)
# 요청 전송 및 토큰 반환
token = send_request_to_api(api_server, model_name, payload)

# 요청 결과 확인
md_header = f'''
## Request
- Request To: `{api_server}/api/v1/task/init` <button onclick="window.open('{api_doc_task_init.format(api_server=api_server)}', '_blank')">API Doc</button>
- Returned Token: `{token}` <button onclick="navigator.clipboard.writeText('{token}')">Copy</button>
### Payload
```json
{json.dumps(payload, indent=4)}
```
'''
display(Markdown(md_header))

# 2. Getting the progress of the task by token from the API server
- 아래 셀을 통해서 API server에서 `token`에 대한 태스크의 진행 상황을 확인할 수 있습니다.
- 태스크가 완료되면 `reports`에 결과가 반환됩니다.
- 아래 셀을 통해서 입력 `token`에 대한 결과를 확인할 수 있습니다.
- `reports_by_token: ReportRecord`의 클래스 구조는 [ReportRecord.py](../kafka_utils/models/ReportRecord.py)를 참조해주세요 

In [ ]:
# See the progress of the `report` topic through the `api/v1/task/progress` endpoint

# Set specific token for testing
# token = 'V7/3ixCOyYyyyvLxzUj4GH3d6v8DIiWQEmTd3LL0hOc='

# 태스크 진행 상황 확인
reports_by_token = get_progress_from_api(api_server, token)

# 결과 확인
if len(reports_by_token.reports) == 0:
    display(Markdown('No reports found'))
else:
    md_header = f'''
## Reports
- Progress From: `{api_server}/api/v1/task/progress` <button onclick="window.open('{api_doc_task_progress.format(api_server=api_server)}', '_blank')">API Doc</button>
- token: `{token}` <button onclick="navigator.clipboard.writeText('{token}')">Copy</button>
- Number of reports: {len(reports_by_token.reports)}
    '''
    display(Markdown(md_header))

    # Display the each of report
    for report_idx, report in reports_by_token.reports.items():
        md_block = f'''
### Report #{report_idx}
- modelName: `{report.model_name}`

```json
{json.dumps(report.payload, indent=4)}
```'''
        display(Markdown(md_block))
